In [3]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"
if "DEEPSEEK_API_KEY" not in os.environ:
    os.environ["DEEPSEEK_API_KEY"] = getpass.getpass(
        prompt="Enter your DeepSeek API key (required if using DeepSeek): "
    )
    
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [5]:
from typing import Optional
from typing_extensions import Annotated, TypedDict

class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke."]
    punchline: Annotated[str, ..., "The punchline of the joke."]
    rating: Annotated[Optional[int], None, "How funny the joke is. from 1 to 10."]
    
structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats.")

{'setup': "Why don't cats play poker in the jungle?",
 'punchline': 'Too many cheetahs!',
 'rating': 7}

In [4]:
from typing import Optional
from pydantic import BaseModel, Field

# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""
    
    setup: str = Field(description="The setup of the joke.")
    punchline: str = Field(description="The punchline of the joke.")
    rating: Optional[int] = Field(
        default=None,
        description="How funny the joke is, from 1 to 10. 1 is not funny at all, 10 is very funny.",
    )
    
structured_llm = llm.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats")

Joke(setup="Why don't cats play poker in the wild?", punchline='Too many cheetahs!', rating=7)

In [7]:
from typing import Union

class Joke(BaseModel):
    """Joke to tell user."""
    
    setup: str = Field(description="The setup of the joke.")
    punchline: str = Field(description="The punchline of the joke.")
    rating: Optional[Union[int, float]] = Field(
        default=None,
        description="How funny the joke is, from 1 to 10. 1 is not funny at all, 10 is very funny.",
    )
    
class ConversationalResponse(BaseModel):
    """Respond in a conversational manner. Be kind and friendly."""
    
    response: str = Field(description="A conversational response to the user's query.")

class FinalResponse(BaseModel):
    final_output: Union[Joke, ConversationalResponse]
    
structured_llm = llm.with_structured_output(FinalResponse)

structured_llm.invoke("Tell me a joke about China")

FinalResponse(final_output=Joke(setup='Why did the panda from China refuse to eat the sandwich?', punchline='Because it was a bamboo-zle!', rating=7))

In [8]:
structured_llm.invoke("中国首都是哪里？")

FinalResponse(final_output=ConversationalResponse(response='中国的首都是北京。'))

In [10]:
from typing_extensions import Annotated, TypedDict

# TypedDict
class Joke(TypedDict):
    """Joke to tell user."""
    
    setup: Annotated[str, ..., "The setup of the joke."]
    punchline: Annotated[str, ..., "The punchline of the joke."]
    rating: Annotated[Optional[int], None, "How funny the joke is. from 1 to 10."]
    
structured_llm = llm.with_structured_output(Joke)

for chunk in structured_llm.stream("Tell me a joke about cats"):
    print(chunk)

{}
{'setup': ''}
{'setup': 'Why'}
{'setup': 'Why did'}
{'setup': 'Why did the'}
{'setup': 'Why did the cat'}
{'setup': 'Why did the cat sit'}
{'setup': 'Why did the cat sit on'}
{'setup': 'Why did the cat sit on the'}
{'setup': 'Why did the cat sit on the computer'}
{'setup': 'Why did the cat sit on the computer?'}
{'setup': 'Why did the cat sit on the computer?', 'punchline': ''}
{'setup': 'Why did the cat sit on the computer?', 'punchline': 'To'}
{'setup': 'Why did the cat sit on the computer?', 'punchline': 'To keep'}
{'setup': 'Why did the cat sit on the computer?', 'punchline': 'To keep an'}
{'setup': 'Why did the cat sit on the computer?', 'punchline': 'To keep an eye'}
{'setup': 'Why did the cat sit on the computer?', 'punchline': 'To keep an eye on'}
{'setup': 'Why did the cat sit on the computer?', 'punchline': 'To keep an eye on the'}
{'setup': 'Why did the cat sit on the computer?', 'punchline': 'To keep an eye on the mouse'}
{'setup': 'Why did the cat sit on the computer?',

In [13]:
from langchain_core.prompts import ChatPromptTemplate

system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") and the final punchline (the response to "<setup> who?").

Here are some examples of jokes:

example_user: Tell me a joke about planes
example_assistant: {{"setup": "Why don't planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}}

example_user: Tell me another joke about planes
example_assistant: {{"setup": "Cargo", "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!", "rating": 10}}

example_user: Now about caterpillars
example_assistant: {{"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}}"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke("Tell me a joke about cats")

{'setup': 'Meow', 'punchline': 'Meow you’re just kitten around!', 'rating': 7}